In [555]:
from __future__ import print_function, division
%matplotlib inline

# import used packages and modules
# import display function to display the table of the pandas dataframe
from IPython.display import display

from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import KFold
from sklearn.model_selection import LeaveOneOut, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier

from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import accuracy_score, confusion_matrix

from scipy import signal

from hmmlearn import hmm
from seqlearn.hmm import MultinomialHMM

import datetime
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import csv


import tensorflow as tf
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from tflearn.layers.normalization import local_response_normalization

# Importing Data

In [556]:
train_manchester = pd.read_csv('../data/misc/Train_Manchester_Northwich.csv')
london_data = pd.read_csv('../data/london_data/london_data.csv')

data04 = pd.read_csv('../data/meadows_december/meadows-2017-12-04.csv')
data05 = pd.read_csv('../data/meadows_december/meadows-2017-12-05.csv')
data06 = pd.read_csv('../data/meadows_december/meadows-2017-12-06.csv')
data07 = pd.read_csv('../data/meadows_december/meadows-2017-12-07.csv')

bike_data_old = pd.read_csv('../data/misc/2015061911.csv')
bike_data_old2 = pd.read_csv('../data/misc/2015061817.csv')
bike_data_old3 = pd.read_csv('../data/misc/20150618.csv')
bike_data_feb = pd.read_csv('../data/misc/bike_feb.csv')

walk_data_vlad1 = pd.read_csv('../data/walking_validation/015Outdoor-Day-2018-02-28.csv')
walk_data_vlad2 = pd.read_csv('../data/walking_validation/016Outdoor-Night-2018-02-28.csv')

# Merging Data

In [557]:
bin_vals = ['bin'+str(x) for x in range(0,16)]
norm_bin_vals = ['bin_norm'+str(x) for x in range(0,16)]
pm_vals = ['pm1', 'pm2_5', 'pm10']

In [558]:
# environment index is 0 as all data from these datasets is walking data
data04['environment_index'] = 0
data05['environment_index'] = 0
data06['environment_index'] = 0
data07['environment_index'] = 0

walk_data_vlad1['environment_index'] = 0
walk_data_vlad2['environment_index'] = 0

# environment index is 3 for bike data
bike_data_old['environment_index'] = 3
bike_data_old2['environment_index'] = 3
bike_data_old3['environment_index'] = 3
bike_data_feb['environment_index'] = 3

# rename columns
bike_data_old2 = bike_data_old2.rename(
    index=str, columns={"latitude":'gpsLatitude', "longitude":'gpsLongitude'})
bike_data_old3 = bike_data_old3.rename(
    index=str, columns={"latitude":'gpsLatitude', "longitude":'gpsLongitude'})

labels = bin_vals + pm_vals + ['environment_index', 'gpsLatitude', 'gpsLongitude', 'humidity', 'temperature']

data = pd.concat([london_data[labels],
                  train_manchester[labels], 
                  data04[labels], 
                  data05[labels],
                  data06[labels],
                  data07[labels],
                  walk_data_vlad1[labels],
                  walk_data_vlad2[labels],
                  bike_data_old[labels],
                  bike_data_old2[labels],
                  bike_data_feb[labels]], ignore_index=True)

# remove indoor labelled data for this set of experiments
data = data[data['environment_index']!=7]
data = data[data['environment_index']!=5]
data = data[data['humidity']>0]
data = data[data['temperature']>0]
data['total'] = data[bin_vals].apply(lambda row: np.sum(row), axis=1)

### Data Information

In [559]:
data.describe()

,bin0,bin1,bin2,bin3,bin4,bin5,bin6,bin7,bin8,bin9,...,bin15,pm1,pm2_5,pm10,environment_index,gpsLatitude,gpsLongitude,humidity,temperature,total
count,4130.000000,4130.000000,4130.000000,4130.000000,4130.000000,4130.000000,4130.000000,4130.000000,4130.000000,4130.000000,...,4130.000000,4130.000000,4130.000000,4130.000000,4130.000000,4130.000000,4130.000000,4130.000000,4130.000000,4130.000000
mean,404.870218,103.113559,61.670944,36.727119,13.663196,12.728329,7.344794,3.639952,4.800484,3.304116,...,8.493947,4.409408,9.475047,21.853523,1.785230,54.734052,-2.622052,53.015767,18.038477,713.140678
std,925.144861,239.027580,136.778940,72.701997,26.822563,25.064795,26.786993,55.166780,156.937704,148.153861,...,371.872862,8.966092,12.446566,54.705197,1.487699,5.749767,1.051517,10.598606,8.018367,3207.199301
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-3.258313,17.600000,0.100000,0.000000
25%,14.000000,5.000000,4.000000,2.000000,2.000000,2.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,1.296981,3.588506,5.644779,0.000000,55.936368,-3.193144,46.200001,13.700000,29.000000
50%,126.000000,44.000000,33.000000,20.000000,7.000000,6.000000,3.000000,1.000000,0.000000,0.000000,...,0.000000,2.237693,6.139762,9.472539,2.000000,55.943145,-3.186589,53.535116,17.725000,270.500000
75%,454.000000,110.000000,72.000000,45.000000,16.000000,16.000000,8.000000,2.000000,1.000000,1.000000,...,0.000000,4.404048,10.942817,19.654435,3.000000,55.945540,-2.519413,57.354206,23.500000,715.000000
max,12336.000000,4758.000000,3710.000000,1997.000000,538.000000,434.000000,656.000000,3343.000000,7955.000000,9500.000000,...,16999.000000,108.471610,158.405240,1051.750977,6.000000,56.008247,-0.100202,99.800000,110.700005,152492.000000


### Labels

In [560]:
labels_idx = np.array([0, 1, 2, 3, 4, 6])

# Kmeans

In [561]:
# method to get the cluster means of the data points based on coordinates
# and the queried columns
def get_location_cluster_means(data, cluster_no, cols):
    # obtain indices of clusters
    kmeans = KMeans(n_clusters=cluster_no, random_state=0)
    # remove data rows with null coordinates
    data = data[~np.isnan(data['gpsLatitude'])]
    # get the indices of the location based clusters
    indices = kmeans.fit_predict(data[['gpsLatitude', 'gpsLongitude']])
    freqs = np.bincount(indices)
    # initialise means of queried columns
    means = np.zeros((cluster_no, len(cols)))
    # assign clustered_indices to dataframe
    data['clustered_index'] = indices
    # compute means of queried columns
    for index in np.unique(indices):
        means[index] = np.mean(data[data['clustered_index'] == index][cols])
    return means, data

In [562]:
# method to get the clusters of the environments corresponding to the 5 environments based on
# the queried columns and number of location clusters
def get_environment_clusters(data, cluster_no, cols):
    means, d = get_location_cluster_means(data, cluster_no, cols)
    # we have 5 clusters corresponding to 6 different types of transport
    environment_kmeans = KMeans(n_clusters=6, random_state=0)
    # predict the cluster indices
    environment_indices = environment_kmeans.fit_predict(means)
    # sort the indices based on the means of the clusters
    idx = np.argsort(environment_kmeans.cluster_centers_.sum(axis=1))
    lut = np.zeros_like(idx)
    lut[idx] = np.arange(6)
    # append the indices to the dataframe
    d['unsupervised_environment_index'] = lut[environment_indices][d['clustered_index']]
    return d

# Stats Methods

In [563]:
def plot_confusion_matrix(labels, predictions, title='Confusion matrix', 
                          classes=['on_foot', 'car', 'train', 'bike', 'bus', 'underground']):
    """Plots a confusion matrix."""
    cmatrix = np.array(confusion_matrix(labels, predictions))
    # normalize confusion matrix
    cm = cmatrix/cmatrix.sum(axis=1)[:, np.newaxis]
    if classes is not None:
        sns.heatmap(cm, xticklabels=classes, yticklabels=classes, vmin=0., vmax=1., annot=True)
    else:
        sns.heatmap(cm, vmin=0., vmax=1.)
    plt.title(title)
    plt.ylabel('True data')
    plt.xlabel('Predicted data')
    plt.show()

In [564]:
def get_training_accuracy_with_kfolds_iloc(estimator, x_tr, y_tr, kf):
    score_array = np.empty(kf.n_folds)
    
    for (idx, (train_feature, test_feature)) in enumerate(kf):
        estimator.fit(x_tr.iloc[train_feature], y_tr.iloc[train_feature])
        y_pred = estimator.predict(x_tr.iloc[test_feature])
        cm = confusion_matrix(y_tr.iloc[test_feature], y_pred)
        cm_norm = cm.astype('float')/cm.sum(axis=1)
        score_array[idx] = accuracy_score(y_tr.iloc[test_feature], y_pred, normalize=True)
        
#         plot_confusion_matrix(y_tr.iloc[test_feature], y_pred)
        
    return np.mean(score_array)

In [565]:
def get_training_accuracy_mixed_models(estimators, x_tr, y_tr, kf):
    score_array = np.empty(kf.n_folds)
    
    for (idx, (train_feature, test_feature)) in enumerate(kf):
        probs_array = np.empty((len(estimators), x_tr.iloc[test_feature].shape[0], 6))

        for estimator_idx, estimator in enumerate(estimators):
            estimator.fit(x_tr.iloc[train_feature], y_tr.iloc[train_feature])
            probs_array[estimator_idx] = estimator.predict_proba(x_tr.iloc[test_feature])
        
        final_labels = labels_idx[np.argmax(np.mean(probs_array, axis=0), axis=1)]
        score_array[idx] = accuracy_score(y_tr.iloc[test_feature], final_labels, normalize=True)
    
    return np.mean(score_array)

# Absolute Values

### Bin Values

In [566]:
svc = SVC()
kf = KFold(len(data), n_folds=5, shuffle=True, random_state=0)

print("Accuracy:", get_training_accuracy_with_kfolds_iloc(svc, data[bin_vals], 
                                                          data['environment_index'], kf))

Accuracy: 0.7104116222760289


In [567]:
rf = RandomForestClassifier(random_state=0)
kf = KFold(len(data), n_folds=5, shuffle=True, random_state=0)

print("Accuracy:", get_training_accuracy_with_kfolds_iloc(rf, data[bin_vals], 
                                                          data['environment_index'], kf))

Accuracy: 0.923728813559322


In [568]:
lr = LogisticRegressionCV(n_jobs=-1)
kf = KFold(len(data), n_folds=5, shuffle=True, random_state=0)

print("Accuracy:", get_training_accuracy_with_kfolds_iloc(lr, data[bin_vals], 
                                                          data['environment_index'], kf))

Accuracy: 0.8397094430992735


In [569]:
knn = KNeighborsClassifier(n_neighbors=5)
kf = KFold(len(data), n_folds=5, shuffle=True, random_state=0)

print("Accuracy:", get_training_accuracy_with_kfolds_iloc(knn, data[bin_vals], 
                                                          data['environment_index'], kf))

Accuracy: 0.9029055690072638


In [570]:
clf = MLPClassifier()
kf = KFold(len(data), n_folds=5, shuffle=True, random_state=0)

print("Accuracy:", get_training_accuracy_with_kfolds_iloc(clf, data[bin_vals], 
                                                          data['environment_index'], kf))

Accuracy: 0.8157384987893461


### First 3 Bin Counts

In [571]:
svc = SVC()
kf = KFold(len(data), n_folds=5, shuffle=True, random_state=0)

print("Accuracy:", get_training_accuracy_with_kfolds_iloc(svc, data[bin_vals[:3]], 
                                                          data['environment_index'], kf))

Accuracy: 0.708958837772397


In [572]:
rf = RandomForestClassifier(random_state=0)
kf = KFold(len(data), n_folds=5, shuffle=True, random_state=0)

print("Accuracy:", get_training_accuracy_with_kfolds_iloc(rf, data[bin_vals[:3]], 
                                                          data['environment_index'], kf))

Accuracy: 0.8963680387409202


In [573]:
lr = LogisticRegressionCV(n_jobs=-1)
kf = KFold(len(data), n_folds=5, shuffle=True, random_state=0)

print("Accuracy:", get_training_accuracy_with_kfolds_iloc(lr, data[bin_vals[:3]], 
                                                          data['environment_index'], kf))

Accuracy: 0.7973365617433414


In [574]:
knn = KNeighborsClassifier(n_neighbors=5)
kf = KFold(len(data), n_folds=5, shuffle=True, random_state=0)

print("Accuracy:", get_training_accuracy_with_kfolds_iloc(knn, data[bin_vals[:3]], 
                                                          data['environment_index'], kf))

Accuracy: 0.8903147699757868


In [575]:
clf = MLPClassifier()
kf = KFold(len(data), n_folds=5, shuffle=True, random_state=0)

print("Accuracy:", get_training_accuracy_with_kfolds_iloc(clf, data[bin_vals[:3]], 
                                                          data['environment_index'], kf))

Accuracy: 0.7966101694915253


### PM Values

In [576]:
svc = SVC()
kf = KFold(len(data), n_folds=5, shuffle=True, random_state=0)

print("Accuracy:", get_training_accuracy_with_kfolds_iloc(svc, data[pm_vals], 
                                                          data['environment_index'], kf))

Accuracy: 0.8319612590799033


In [577]:
rf = RandomForestClassifier(random_state=0)
kf = KFold(len(data), n_folds=5, shuffle=True, random_state=0)

print("Accuracy:", get_training_accuracy_with_kfolds_iloc(rf, data[pm_vals], 
                                                          data['environment_index'], kf))

Accuracy: 0.8750605326876514


In [578]:
knn = KNeighborsClassifier(n_neighbors=5)
kf = KFold(len(data), n_folds=5, shuffle=True, random_state=0)

print("Accuracy:", get_training_accuracy_with_kfolds_iloc(knn, data[pm_vals], 
                                                          data['environment_index'], kf))

Accuracy: 0.8433414043583536


In [579]:
lr = LogisticRegressionCV(n_jobs=-1)
kf = KFold(len(data), n_folds=5, shuffle=True, random_state=0)

print("Accuracy:", get_training_accuracy_with_kfolds_iloc(lr, data[pm_vals], 
                                                          data['environment_index'], kf))

Accuracy: 0.7675544794188862


In [580]:
clf = MLPClassifier()
kf = KFold(len(data), n_folds=5, shuffle=True, random_state=0)

print("Accuracy:", get_training_accuracy_with_kfolds_iloc(clf, data[pm_vals], 
                                                          data['environment_index'], kf))

Accuracy: 0.7939467312348667


### Urban Environments

In [642]:
clustered_data = get_environment_clusters(data, 40, bin_vals)

rf = RandomForestClassifier(random_state=0, n_jobs=-1)
kf = KFold(len(clustered_data), n_folds=5, shuffle=True, random_state=0)

print("Accuracy:", get_training_accuracy_with_kfolds_iloc(rf, clustered_data[['unsupervised_environment_index']+bin_vals], 
                                                          clustered_data['environment_index'], kf))

Accuracy: 0.949636803874092


In [582]:
clustered_data = get_environment_clusters(data, 40, bin_vals)

svc = SVC()
kf = KFold(len(clustered_data), n_folds=5, shuffle=True, random_state=0)

print("Accuracy:", get_training_accuracy_with_kfolds_iloc(svc, clustered_data[['unsupervised_environment_index']+bin_vals], 
                                                          clustered_data['environment_index'], kf))

Accuracy: 0.7113801452784504


In [583]:
clustered_data = get_environment_clusters(data, 40, bin_vals)

lr = LogisticRegressionCV(n_jobs=-1)
kf = KFold(len(clustered_data), n_folds=5, shuffle=True, random_state=0)

print("Accuracy:", get_training_accuracy_with_kfolds_iloc(lr, clustered_data[['unsupervised_environment_index']+bin_vals], 
                                                          clustered_data['environment_index'], kf))

Accuracy: 0.8648910411622275


In [584]:
clustered_data = get_environment_clusters(data, 40, bin_vals)

knn = KNeighborsClassifier()
kf = KFold(len(clustered_data), n_folds=5, shuffle=True, random_state=0)

print("Accuracy:", get_training_accuracy_with_kfolds_iloc(knn, clustered_data[['unsupervised_environment_index']+bin_vals], 
                                                          clustered_data['environment_index'], kf))

Accuracy: 0.9031476997578691


In [585]:
clustered_data = get_environment_clusters(data, 40, bin_vals)

clf = MLPClassifier()
kf = KFold(len(clustered_data), n_folds=5, shuffle=True, random_state=0)

print("Accuracy:", get_training_accuracy_with_kfolds_iloc(clf, clustered_data[['unsupervised_environment_index']+bin_vals], 
                                                          clustered_data['environment_index'], kf))

Accuracy: 0.8445520581113801


### Normalised Bin Counts

In [586]:
data_normalised = data[data['total']>0].copy().reindex()
data_normalised[bin_vals] = data_normalised[bin_vals].apply(lambda row: row/np.sum(row), axis=1)
data_normalised['pca_pm'] = PCA(n_components=1).fit_transform(data_normalised[pm_vals])

In [617]:
rf = RandomForestClassifier(random_state=0)
kf = KFold(len(data_normalised), n_folds=5, shuffle=True, random_state=0)

print("Accuracy:", get_training_accuracy_with_kfolds_iloc(rf, data_normalised[bin_vals], 
                                                          data_normalised['environment_index'], kf))

Accuracy: 0.888615475139747


In [588]:
svc = SVC()
kf = KFold(len(data_normalised), n_folds=5, shuffle=True, random_state=0)

print("Accuracy:", get_training_accuracy_with_kfolds_iloc(svc, data_normalised[bin_vals], 
                                                          data_normalised['environment_index'], kf))

Accuracy: 0.5792291850544278


In [616]:
knn = KNeighborsClassifier(n_neighbors=5)
kf = KFold(len(data_normalised), n_folds=5, shuffle=True, random_state=0)

print("Accuracy:", get_training_accuracy_with_kfolds_iloc(knn, data_normalised[bin_vals], 
                                                          data_normalised['environment_index'], kf))

Accuracy: 0.7993186231244485


In [615]:
clf = MLPClassifier()
kf = KFold(len(data_normalised), n_folds=5, shuffle=True, random_state=0)

print("Accuracy:", get_training_accuracy_with_kfolds_iloc(clf, data_normalised[bin_vals], 
                                                          data_normalised['environment_index'], kf))

/Users/mihaivisuian/miniconda3/envs/hons/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Accuracy: 0.7816081200353044


# Best Models

In [640]:
clustered_data = get_environment_clusters(data, 40, bin_vals)

clustered_normalised_data = clustered_data[clustered_data['total']>0].copy().reindex()
clustered_normalised_data[bin_vals] = clustered_normalised_data[bin_vals].apply(lambda row: row/np.sum(row), axis=1)

rf = RandomForestClassifier(random_state=0, n_estimators=260, n_jobs=-1)
kf = KFold(len(clustered_normalised_data), n_folds=5, shuffle=True, random_state=0)

print("Accuracy:", get_training_accuracy_with_kfolds_iloc(rf, clustered_normalised_data[['unsupervised_environment_index', 'total']+bin_vals], 
                                                          clustered_normalised_data['environment_index'], kf))

Accuracy: 0.9565631067961166


## Add temperature and humidity

In [636]:
clustered_data = get_environment_clusters(data, 40, bin_vals)

clustered_normalised_data = clustered_data[clustered_data['total']>0].copy().reindex()
clustered_normalised_data[bin_vals] = clustered_normalised_data[bin_vals].apply(lambda row: row/np.sum(row), axis=1)

rf = RandomForestClassifier(random_state=0, n_estimators=260, n_jobs=-1)
kf = KFold(len(clustered_normalised_data), n_folds=5, shuffle=True, random_state=0)

print("Accuracy:", get_training_accuracy_with_kfolds_iloc(rf, clustered_normalised_data[['unsupervised_environment_index', 'temperature', 'humidity', 'total']+bin_vals], 
                                                          clustered_normalised_data['environment_index'], kf))

Accuracy: 0.9703951162106502


In [627]:
clustered_data = get_environment_clusters(data, 40, bin_vals)

clustered_normalised_data = clustered_data[clustered_data['total']>0].copy().reindex()
clustered_normalised_data[bin_vals] = clustered_normalised_data[bin_vals].apply(lambda row: row/np.sum(row), axis=1)

knn = KNeighborsClassifier()
kf = KFold(len(clustered_normalised_data), n_folds=5, shuffle=True, random_state=0)

print("Accuracy:", get_training_accuracy_with_kfolds_iloc(knn, clustered_normalised_data[['unsupervised_environment_index', 'total']+bin_vals], 
                                                          clustered_normalised_data['environment_index'], kf))

Accuracy: 0.8349891144454251


### Neural Network

In [613]:
# environment clustering on raw bin values: 3 hidden layers
clustered_data = get_environment_clusters(data, 40, bin_vals)

clustered_normalised_data = clustered_data[clustered_data['total']>0].copy().reindex()
clustered_normalised_data[bin_vals] = clustered_normalised_data[bin_vals].apply(lambda row: row/np.sum(row), axis=1)

clf = MLPClassifier(solver='lbfgs', 
                    alpha=1e-5, 
                    hidden_layer_sizes=(20,), 
                    random_state=1, 
                    max_iter=500)
kf = KFold(len(clustered_normalised_data), n_folds=5, shuffle=True, random_state=0)

print("Accuracy:", get_training_accuracy_with_kfolds_iloc(clf, clustered_normalised_data[['unsupervised_environment_index']+bin_vals], 
                                                          clustered_normalised_data['environment_index'], kf))

Accuracy: 0.8490717858193587


### KNN

In [628]:
clustered_data = get_environment_clusters(data, 40, bin_vals)

clustered_normalised_data = clustered_data[clustered_data['total']>0].copy().reindex()
clustered_normalised_data[bin_vals] = clustered_normalised_data[bin_vals].apply(lambda row: row/np.sum(row), axis=1)

knn = KNeighborsClassifier(n_neighbors=5, n_jobs=-1)
kf = KFold(len(clustered_normalised_data), n_folds=5, shuffle=True, random_state=0)

print("Accuracy:", get_training_accuracy_with_kfolds_iloc(knn, clustered_normalised_data[['unsupervised_environment_index']+bin_vals], 
                                                          clustered_normalised_data['environment_index'], kf))

Accuracy: 0.8439720506031186


### Mixed Model

In [633]:
kf = KFold(len(clustered_normalised_data), n_folds=5, shuffle=True, random_state=0)

get_training_accuracy_mixed_models([knn, rf, clf], clustered_normalised_data[['unsupervised_environment_index', 'humidity', 'temperature']+bin_vals], 
                                                          clustered_normalised_data['environment_index'], kf)

0.9728214180641365

### Validating Models

#### Train Data

In [597]:
clustered_train = get_environment_clusters(data, 40, bin_vals).reindex()
clustered_normalised_train = clustered_train[clustered_train['total']>0].copy().reindex()
clustered_normalised_train[bin_vals] = clustered_normalised_train[bin_vals].apply(lambda row: row/np.sum(row), axis=1)

#### Bike validation data

In [598]:
bike_data_validation = pd.read_csv('../data/misc/2015072817.csv')
bike_data_validation['environment_index'] = 3
bike_data_validation['total'] = bike_data_validation[bin_vals].apply(lambda row: np.sum(row), axis=1)

bike_data_validation = bike_data_validation.rename(index=str, columns={"latitude":'gpsLatitude', "longitude":'gpsLongitude'})
display(bike_data_validation.describe())

,time,gpsLatitude,gpsLongitude,bin0,bin1,bin2,bin3,bin4,bin5,bin6,...,temperature,humidity,alphaVersion,urban_space,o3_ae,o3_we,no2_ae,no2_we,environment_index,total
count,2.500000e+02,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,...,250.000000,250.000000,250.0,250.0,250.000000,250.000000,250.000000,250.000000,250.0,250.000000
mean,2.015073e+13,55.946380,-3.189356,3.016000,0.344000,0.220000,0.168000,0.136000,0.240000,0.188000,...,23.601040,49.532016,2.0,0.0,-25647.920000,-25242.416000,-27040.560000,-26069.808000,3.0,4.628000
std,1.221088e+02,0.000037,0.000111,1.838823,0.575316,0.452126,0.405504,0.354976,0.528695,0.421148,...,0.606837,4.011418,0.0,0.0,6289.142903,3210.032059,5074.136107,3632.280436,0.0,2.217495
min,2.015073e+13,55.946215,-3.189550,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,21.960000,44.655179,2.0,0.0,-32222.000000,-31198.000000,-32734.000000,-32478.000000,3.0,0.000000
25%,2.015073e+13,55.946380,-3.189470,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,23.342500,45.051665,2.0,0.0,-28638.000000,-27358.000000,-29918.000000,-28638.000000,3.0,3.000000
50%,2.015073e+13,55.946398,-3.189300,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,23.755000,49.939463,2.0,0.0,-26846.000000,-25822.000000,-27742.000000,-26334.000000,3.0,4.000000
75%,2.015073e+13,55.946398,-3.189300,4.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,24.067500,51.480962,2.0,0.0,-25054.000000,-24286.000000,-25310.000000,-24542.000000,3.0,6.000000
max,2.015073e+13,55.946398,-3.189151,10.000000,3.000000,2.000000,2.000000,2.000000,3.000000,2.000000,...,24.180000,58.565045,2.0,0.0,32546.000000,-13534.000000,32034.000000,-11742.000000,3.0,12.000000


In [599]:
clustered_validation_data = get_environment_clusters(bike_data_validation, 40, bin_vals+['total'])

clustered_normalised = clustered_validation_data[clustered_validation_data['total']>0].copy().reindex()
clustered_normalised[bin_vals] = clustered_normalised[bin_vals].apply(lambda row: row/np.sum(row), axis=1)

rf.fit(clustered_normalised_train[bin_vals+['unsupervised_environment_index', 'total']], 
       clustered_normalised_train['environment_index'])

predictions = rf.predict(clustered_normalised[bin_vals+['unsupervised_environment_index', 'total']])

print("Validation accuracy:", accuracy_score(clustered_normalised['environment_index'], predictions))
 

Validation accuracy: 1.0


In [600]:
clustered_validation_data = get_environment_clusters(bike_data_validation, 40, bin_vals+['total'])

rf.fit(clustered_train[bin_vals + ['unsupervised_environment_index', 'total']], 
       clustered_train['environment_index'])

predictions = rf.predict(clustered_validation_data[bin_vals + ['unsupervised_environment_index', 'total']])

print("Validation accuracy:", accuracy_score(clustered_validation_data['environment_index'], predictions))
 

Validation accuracy: 0.984


### Walking Validation Data

In [601]:
walk_data_validation1 = pd.read_csv('../data/walking_validation/015Outdoor-Day-2018-02-28.csv')
walk_data_validation2 = pd.read_csv('../data/walking_validation/016Outdoor-Night-2018-02-28.csv')
walk_data_validation3 = pd.read_csv('../data/walking_validation/024Outdoor-Day-2018-03-05.csv')

walk_data_validation_total = pd.concat([walk_data_validation3], ignore_index=True)

walk_data_validation_total['environment_index'] = 0

display(walk_data_validation_total.describe())

,phoneTimestamp,pm1,pm2_5,pm10,temperature,humidity,bin0,bin1,bin2,bin3,...,bin14,bin15,total,gpsLongitude,gpsLatitude,gpsAltitude,gpsAccuracy,luxLevel,motion,environment_index
count,2.860000e+02,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,...,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.0
mean,1.520262e+12,10.094618,12.424445,17.979335,6.921679,64.987413,539.772727,101.111888,40.804196,17.562937,...,0.017483,0.024476,720.342657,-3.187221,55.951014,127.997114,10.024007,31.576923,3246.398601,0.0
std,2.287221e+06,7.186586,8.985549,11.127342,3.367724,20.525194,330.817500,106.139141,50.443404,21.009803,...,0.131290,0.154791,493.213247,0.001629,0.003453,14.690534,0.193646,18.184996,741.746731,0.0
min,1.520260e+12,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,-3.191518,55.944679,71.707644,10.000000,4.000000,39.000000,0.0
25%,1.520260e+12,6.068395,7.532011,10.323843,5.600000,63.100002,326.000000,57.000000,23.000000,10.000000,...,0.000000,0.000000,432.000000,-3.188255,55.947735,120.419377,10.000000,19.000000,2887.000000,0.0
50%,1.520261e+12,7.997307,9.873313,16.104126,6.000000,73.700005,409.000000,76.000000,30.000000,14.000000,...,0.000000,0.000000,556.500000,-3.187173,55.951773,128.613202,10.000000,29.000000,3269.500000,0.0
75%,1.520265e+12,12.648389,15.113871,23.184410,7.700000,76.350001,699.000000,115.000000,47.000000,20.000000,...,0.000000,0.000000,912.500000,-3.186362,55.953629,137.060204,10.000000,39.000000,3559.000000,0.0
max,1.520266e+12,78.974530,103.687230,113.550224,18.100000,84.600000,2995.000000,1311.000000,638.000000,286.000000,...,1.000000,1.000000,5372.000000,-3.183764,55.956490,154.825047,12.883000,111.000000,8304.000000,0.0


In [602]:
clustered_validation_data = get_environment_clusters(walk_data_validation_total, 40, bin_vals)

clustered_normalised = clustered_validation_data[clustered_validation_data['total']>0].copy().reindex()
clustered_normalised[bin_vals] = clustered_normalised[bin_vals].apply(lambda row: row/np.sum(row), axis=1)

rf.fit(clustered_normalised_train[bin_vals+['unsupervised_environment_index', 'temperature', 'humidity', 'total']], 
       clustered_normalised_train['environment_index'])

predictions = rf.predict(clustered_normalised[bin_vals+['unsupervised_environment_index', 'temperature', 'humidity', 'total']])

print("Validation accuracy:", accuracy_score(clustered_normalised['environment_index'], predictions))


Validation accuracy: 0.901060070671378


In [603]:
clustered_validation_data = get_environment_clusters(walk_data_validation_total, 40, bin_vals)

rf.fit(clustered_train[bin_vals + ['temperature', 'humidity', 'unsupervised_environment_index', 'total']], 
       clustered_train['environment_index'])

predictions = rf.predict(clustered_validation_data[bin_vals + ['temperature', 'humidity', 'unsupervised_environment_index', 'total']])

print("Validation accuracy:", accuracy_score(clustered_validation_data['environment_index'], predictions))
 

Validation accuracy: 0.6013986013986014
